In [1]:
from agno.agent import Agent
from agno.models.ollama import Ollama
from pydantic import BaseModel
from pathlib import Path
from agno.reranker.base import Reranker
from agno.document.chunking.agentic import AgenticChunking
from agno.knowledge.document import DocumentKnowledgeBase, Document
from agno.knowledge.combined import CombinedKnowledgeBase
from agno.knowledge.text import TextKnowledgeBase
from agno.knowledge.docx import DocxKnowledgeBase, DocxReader
from agno.knowledge.pdf import PDFKnowledgeBase, PDFReader
from agno.embedder.ollama import OllamaEmbedder
from agno.vectordb.lancedb import LanceDb
from agno.vectordb.search import SearchType
from agno.tools.file import FileTools
from agno.tools.duckduckgo import DuckDuckGoTools

# ollama pull openhermes llama3.1

In [2]:

vector_db = LanceDb(
    table_name="programs",
    uri="../tmp/lancedb",
    search_type=SearchType.hybrid,
    embedder=OllamaEmbedder()
)

chunking = AgenticChunking()

In [3]:
pdf_knowledge_base = PDFKnowledgeBase(
    path="../data/input",
    vector_db = vector_db
)

docx_knowledge_base = DocxKnowledgeBase(
    path="../data/input",
    vector_db = vector_db
)

txt_knowledge_base = TextKnowledgeBase(
    path="../data/input",
    vector_db = vector_db
)

knowledge_base = CombinedKnowledgeBase(
    sources=[
        pdf_knowledge_base,
        docx_knowledge_base,
        txt_knowledge_base,
    ],
    chunking_strategy=chunking,
    vector_db = vector_db
)


In [4]:
# knowledge_base.load(recreate=True)

In [5]:
for i in knowledge_base.document_lists:
    print(i)

INFO     Reading: Analisis y diseño de algoritmos

[Document(content='Análisis y diseño de algoritmos (3009430)', id='Analisis y diseño de algoritmos_1_1', name='Analisis y diseño de algoritmos', meta_data={'page': 1, 'chunk': 1, 'chunk_size': 41}, embedder=None, embedding=None, usage=None, reranking_score=None), Document(content='Objetivo del curso (oficial) Cualquiera con una experiencia mínima en algoritmos y programación sabe que para un mismo problema pueden existir innumerables algoritmos de solución. Tales soluciones pueden variar en diversos aspectos, siendo de particular interés la eficiencia en términos de su costo computacional. En el contexto de dicha eficiencia se puede hablar de algoritmos "mejores" que otros, siendo el objetivo principal de esta asignatura brindarle elementos al estudiante para analizar y diseñar algoritmos eficientes.', id='Analisis y diseño de algoritmos_2_1', name='Analisis y diseño de algoritmos', meta_data={'page': 2, 'chunk': 1, 'chunk_size': 529}, embedder=None, embedding=None, usage=None, reranki

INFO     Reading: Fundamentos de programacion

[Document(content='PRESENTACIÓN Y PROGRAMA DE ASIGNATURA FUNDAMENTOS DE PROGRAMACIÓN Juan Felipe Muñoz Fernández – jfmunozf@unal.edu.co • Código: 3010435 • Nombre: Fundamentos de programación • Grupos: 22, 23 y 24 • Número de créditos: 3 (Ingeniería de sistemas e informática) • Metodología ▪ Sesiones magistrales presenciales los jueves de 14:00 a 16:00 en el aula 302 del bloque 12 del núcleo El Volador. ▪ Sesiones prácticas presenciales los viernes en aula 204 del bloque 4 del núcleo del río: ▪ Grupo 22: viernes de 12:00 a 14:00 ▪ Grupo 23: viernes de 14:00 a 16:00 ▪ Grupo 24: viernes de 16:00 a 18:00 ▪ Asesorías por parte del docente en el horario de clase: jueves y viernes. ▪ Asesorías de monitores y auxiliares en el horario de las sesiones prácticas. • Temas del curso Unidad 1: Introducción ¿Qué es un algoritmo? ¿Qué es un programa de computadora y para qué sirve? ¿Por qué programar en Python? ¿Qué entornos de desarrollo se Python se pueden usar? Primer programa en Python, concepto 

In [6]:
description = "You are an academic content generation assistant. Based on the following university course syllabus, generate structured teaching materials, ensuring clarity, engagement, and alignment with best pedagogical practices. This material are going to be used by a professor to teach a course in [subject]."
instructions = [
    "List the files in the folder '../data/input' and determine if one of this names is usefull for the response.",
    "If one of the files is usefull, read it and use it to know the course topic, else search in internet.",
    "If the topic is specific, ensure accuracy and up-to-date information. If it is general, prioritize relevance and engagement.",
    "If sufficient information is not available, search online for reliable sources before generating content.",
    "Follow these guidelines based on the content type:",
    "- Exam: Generate a set of 5 to 10 exam questions with detailed solutions.",
    "- Lecture: Create a structured lecture outline with at least 5 sections covering key concepts.",
    "- Workshop: Develop a workshop plan with at least 5 sections, including practical examples and exercises.",
    "Ensure clarity, coherence, and academic rigor in all content.",
    "Deliver the content in a professional and engaging manner."
]


In [7]:
agent = Agent(
    model=Ollama(id="llama3.1"),
    description=description,
    instructions=instructions,
    tools=[
        FileTools(), 
        DuckDuckGoTools(news = False)
        ],
    # knowledge=knowledge_base, 
    show_tool_calls=True,
    search_knowledge=True,
    # reasoning=True, 
    structured_outputs=True,
    add_references=True,
    debug_mode=True
    )

In [ ]:
def query(course_name: str, content_type: str, topic: str = None):
    
        
    if topic:
        request = f"""Analyze the subject '{course_name}' and identify the most relevant topics in the folder '../data/input'.
            Based on these topics, generate a content type '{content_type}' materials about the course search and explain them.
            Explain the topic '{topic}', provide definitions, and explain what each thing is.
            Ensure to save the output in a file in the folder '../data/output' with a clear and descriptive filename."""
    else:
        request = f"""Analyze the subject '{course_name}' and identify the most relevant topics covered in a standard syllabus.
        Based on these topics, generate a content type '{content_type}' materials about the course search and explain them.
        Explain each topic, provide definitions, and explain what each thing is.
        Ensure to save the output in a file in the folder '../data/output' with a clear and descriptive filename."""
    return agent.print_response(
        request,
        markdown=True
    )

query("Fundamentos de programacion", "Lecture", "Variables")

DEBUG    *********** Agent ID: d87fe41f-734b-4d5a-8163-65a1a2649068 ***********

DEBUG    *********** Session ID: ecde47da-0429-49bc-8801-1fb3983a8823 ***********

DEBUG    *********** Agent Run Start: 1955fa19-f052-4385-9f29-4ab81254560d ***********

DEBUG    Processing tools for model

DEBUG    Included function save_file from file_tools

...

DEBUG    ============== tool ==============

DEBUG    Error reading file: [Errno 2] No such file or directory: 'c:\\Users\\JUAN\\Documents\\Universidad         
         2024-2\\Redes neuronales\\Repos\\Agents\\notebooks\\..\\data\\input\\introduction_to_programming.pdf'

DEBUG    **************** METRICS ****************

DEBUG    * Time:                        0.0091s

DEBUG    **************** METRICS ******************

In [ ]:
query("Fundamentos de programacion", "Notas de clase", "Python")

DEBUG    *********** Agent ID: db07da6e-a3fb-4ae2-a2a6-c72d120b45f1 ***********

DEBUG    *********** Session ID: b2c0b506-ebfe-4b87-b826-106d613586ba ***********

DEBUG    *********** Agent Run Start: 573413be-df1c-4455-86ea-bcac00746eff ***********

DEBUG    Time to get references: 0.0000s

DEBUG    ---------- Ollama Response Start ----------

...

DEBUG    ---------- Ollama Response End ----------

DEBUG    Added 7 Messages to AgentMemory

DEBUG    Added AgentRun to AgentMemory

DEBUG    --**-- Logging Agent Run

DEBUG    *********** Agent Run End: 573413be-df1c-4455-86ea-bcac00746eff ***********

In [ ]:
query("Analisis y diseño de algoritmos", "Exam")

DEBUG    *********** Agent ID: db07da6e-a3fb-4ae2-a2a6-c72d120b45f1 ***********

DEBUG    *********** Session ID: b2c0b506-ebfe-4b87-b826-106d613586ba ***********

DEBUG    *********** Agent Run Start: 12886afb-e84a-431f-a586-916ae08cb0a8 ***********

DEBUG    Time to get references: 0.0000s

DEBUG    ---------- Ollama Response Start ----------

DEBUG    ============== system ==============

DEBUG    You are an academic content generation assistant. Based on the following university course syllabus,      
         generate structured teaching materials, ensuring clarity, engagement, and alignment with best pedagogical 
         practices. This material are going to be used by a professor to teach a course in [subject].              
                                                                                                                   
         <instructions>                                                                                            
         - List the files in the folder '../data/input' and determine if one of this names is usefull for the      
         response.                                                                                                 
         - If one of the files is usefull, read it and use it to know the course topic, else search in internet.   
         - If the topic is specific, ensure accuracy and up-to-date information. If it is general, prioritize      
         relevance and engagement.                                                                                 
         - If sufficient information is not available, search online for reliable sources before generating        
         content.                                                                                                  
         - Follow these guidelines based on the content type:                                                      
         - - Exam: Generate a set of 5 to 10 exam questions with detailed solutions.                               
         - - Lecture: Create a structured lecture outline with at least 5 sections covering key concepts.          
         - - Workshop: Develop a workshop plan with at least 5 sections, including practical examples and          
         exercises.                                                                                                
         - Ensure clarity, coherence, and academic rigor in all content.                                           
         - Deliver the content in a professional and engaging manner.                                              
         </instructions>                                                                                           
                                                                                                                   
         <additional_information>                                                                                  
         - Use markdown to format your answers.                                                                    
         </additional_information>

DEBUG    ============== user ==============

DEBUG    Analyze the subject 'Analisis y diseño de algoritmos' and identify the most relevant topics covered in a  
         standard syllabus.                                                                                        
                 Based on these topics, generate a content type 'Exam' materials about the course search and       
         explain them.                                                                                             
                 Explain each topic, provide definitions, and explain what each thing is.                          
                 Ensure to save the output in a file in the folder '../data/output' with a clear and descriptive   
         filename.

Output()